In [1]:
import face_recognition as fr
import cv2
import glob
import numpy as np
import os
import mediapipe as mp
import random
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
#data = "./archive/train/*/"
data = "../lfw/*/"
x = []
y = []
classes = {}
#num_classes = len(glob.glob(os.path.join(data)))
#n = random.randint(0, num_classes)
for idx, folder in enumerate(glob.glob(os.path.join(data))):
    if idx == 2000:
        break
    num_pics = len(glob.glob(os.path.join(folder+"*")))
    if num_pics < 10:
        continue
    use = glob.glob(os.path.join(folder+"*"))[:10]
    person_name = folder.split("\\")[1]
    print(idx, person_name)
    classes[idx] = person_name
    for f in use:
        pic = cv2.imread(f)
        detected_pos = fr.face_locations(pic)
        for i in detected_pos:
            top, right, bottom, left = i
            face_image = pic[top:bottom, left:right]
            try:
                face_image = cv2.resize(face_image, (256, 256))
                face_embedding = fr.face_encodings(face_image)[0]
                x.append(face_embedding)
                y.append(idx)
            except:
                continue
print(len(x))
print(x)

In [93]:
import json 
with open("classes.json", "w") as outfile:
    json.dump(classes, outfile)

In [87]:
import json
class_f = open('classes.json')
classes = json.load(class_f)
print(dict(classes))

{'17': 'Abdullah_Gul', '50': 'Adrien_Brody', '121': 'Alejandro_Toledo', '204': 'Alvaro_Uribe', '223': 'Amelie_Mauresmo', '272': 'Andre_Agassi', '290': 'Andy_Roddick', '302': 'Angelina_Jolie', '317': 'Anna_Kournikova', '335': 'Ann_Veneman', '370': 'Ariel_Sharon', '374': 'Ari_Fleischer', '386': 'Arnold_Schwarzenegger', '417': 'Atal_Bihari_Vajpayee', '544': 'Bill_Clinton', '551': 'Bill_Gates', '562': 'Bill_McBride', '573': 'Bill_Simon', '706': 'Britney_Spears', '781': 'Carlos_Menem', '783': 'Carlos_Moya', '831': 'Catherine_Zeta-Jones', '875': 'Charles_Moose', '1047': 'Colin_Powell', '1059': 'Condoleezza_Rice', '1207': 'David_Beckham', '1252': 'David_Nalbandian', '1347': 'Dick_Cheney', '1382': 'Dominique_de_Villepin', '1393': 'Donald_Rumsfeld', '1473': 'Edmund_Stoiber', '1476': 'Eduardo_Duhalde', '1704': 'Fidel_Castro', '1854': 'George_HW_Bush', '1865': 'George_Robertson', '1870': 'George_W_Bush', '1891': 'Gerhard_Schroeder', '1932': 'Gloria_Macapagal_Arroyo', '1940': 'Gonzalo_Sanchez_de_L

In [101]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from joblib import dump, load


model = OneVsRestClassifier(SVC(kernel='linear', probability=True, verbose=True, max_iter=1000))
model.fit(x, y)
dump(model, 'newmodel.joblib')


[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

['newmodel.joblib']

In [102]:
model = load('newmodel.joblib')

In [103]:
model.classes_

array([  17,   50,  121,  204,  223,  272,  290,  302,  317,  335,  370,
        374,  386,  417,  544,  551,  562,  573,  706,  781,  783,  831,
        875, 1047, 1059, 1207, 1252, 1347, 1382, 1393, 1473, 1476, 1704,
       1854, 1865, 1870, 1891, 1932, 1940, 1945, 1964, 1993])

In [104]:
class_list = list(classes.keys())
print(class_list)

['17', '50', '121', '204', '223', '272', '290', '302', '317', '335', '370', '374', '386', '417', '544', '551', '562', '573', '706', '781', '783', '831', '875', '1047', '1059', '1207', '1252', '1347', '1382', '1393', '1473', '1476', '1704', '1854', '1865', '1870', '1891', '1932', '1940', '1945', '1964', '1993']


In [106]:
data = "./archive/new_val/*/"
#data = "../lfw/*/"
for idx, folder in enumerate(glob.glob(os.path.join(data))):
    use = glob.glob(os.path.join(folder+"*"))
    person_name = folder.split("\\")[1]
    print(idx, person_name)
    #classes[idx] = person_name
    for f in use:
        pic = cv2.imread(f)
        detected_pos = fr.face_locations(pic)
        for i in detected_pos:
            top, right, bottom, left = i
            face_image = pic[top:bottom, left:right]
            try:
                face_image = cv2.resize(face_image, (256, 256))
                face_embedding = fr.face_encodings(face_image)[0]
                test = np.asarray(face_embedding)
                test=  np.reshape(test, (1, -1))
                pred = model.predict_proba(test)[0]
                if np.max(pred) > 0.5:
                    print(classes[str(class_list[np.argmax(pred)])], np.max(pred))
                else:
                    #added to see what it predicts when highest probability is quite low
                    print(classes[str(class_list[np.argmax(pred)])], np.max(pred)) 
            except Exception as e:
                raise(e)
                continue
            

0 Angelina_Jolie
Angelina_Jolie 0.8723421696080942
Angelina_Jolie 0.8295308021006402
1 Arnold_Schwarzenegger
Arnold_Schwarzenegger 0.7972873947966037
2 Atal_Bihari_Vajpayee
Atal_Bihari_Vajpayee 0.7543001325087304
Atal_Bihari_Vajpayee 0.8544449815358515
3 Bill_Clinton
Bill_Clinton 0.8975208043705579
Bill_Clinton 0.7191735219063841
Bill_Clinton 0.7708728422259497
4 Bill_Gates
Bill_Gates 0.7896890424346855
Bill_Gates 0.8300043244144298
5 Britney_Spears
Britney_Spears 0.8477697238644784
Britney_Spears 0.809027145629636
Britney_Spears 0.860048139814166
6 David_Beckham
David_Beckham 0.4911438521448386
7 Fidel_Castro
Fidel_Castro 0.7739279188787824
